In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import plotly.graph_objects as go

In [2]:
np.random.seed(1)
X = np.random.uniform(-5, 5, 100)
epsilon = np.random.normal(0, 0.5, 100)
y = 3 * X**3 - 2 * X**2 + 5 + epsilon

In [3]:
# Convert the data to PyTorch tensors
X_tensor = torch.tensor(X, dtype=torch.float32).view(-1, 1)  # shape: (100, 1)
y_tensor = torch.tensor(y, dtype=torch.float32).view(-1, 1)  # shape: (100, 1)

In [4]:
# Define the Linear Regression model (for comparison)
class LinearRegressionModel(nn.Module):
    def __init__(self):
        super(LinearRegressionModel, self).__init__()
        self.linear = nn.Linear(1, 1)  # One feature to one output

    def forward(self, x):
        return self.linear(x)
    
linear_model = LinearRegressionModel()

In [5]:
# Define the Polynomial Regression model (degree 3)
class PolynomialRegressionModel(nn.Module):
    def __init__(self):
        super(PolynomialRegressionModel, self).__init__()
        # The input is X, but we'll create polynomial features manually
        self.poly = nn.Linear(4, 1)  # 4 features: X^3, X^2, X, and bias (1)

    def forward(self, x):
        # Create polynomial features manually (X^3, X^2, X, 1)
        x_poly = torch.cat([x**3, x**2, x, torch.ones_like(x)], dim=1)
        return self.poly(x_poly)

poly_model = PolynomialRegressionModel()

In [6]:
# Train the models and track their losses
def train_model(model, X_tensor, y_tensor, num_epochs=1000, learning_rate=0.0003):
    criterion = nn.MSELoss()
    optimizer = optim.SGD(model.parameters(), lr=learning_rate)
    
    loss_values = []

    for _ in range(num_epochs):
        # Forward pass: Compute predicted y by passing X to the model
        y_pred = model(X_tensor)
        loss = criterion(y_pred, y_tensor)
        optimizer.zero_grad()  # Zero the gradients before the backward pass
        loss.backward()  # Backpropagation
        optimizer.step()
        loss_values.append(loss.item())

    return loss_values

In [7]:
# Train both models
linear_loss = train_model(linear_model, X_tensor, y_tensor)
poly_loss = train_model(poly_model, X_tensor, y_tensor)

In [8]:
# Visualize the training loss for both models using Plotly
fig = go.Figure()

# Add traces for both models
fig.add_trace(go.Scatter(x=np.arange(len(linear_loss)), y=linear_loss, mode='lines', name='Linear Regression'))
fig.add_trace(go.Scatter(x=np.arange(len(poly_loss)), y=poly_loss, mode='lines', name='Polynomial Regression (degree 3)'))

# Customize the layout
fig.update_layout(
    title="Training Loss for Linear and Polynomial Regression Models",
    xaxis=dict(title="Epochs"),
    yaxis=dict(title="Loss (MSE)"),
    showlegend=True
)

# Show the plot
fig.show()

# 8. Output the learned parameters for both models
print("Linear Regression Model Parameters:")
for param in linear_model.parameters():
    print(param)

print("\nPolynomial Regression Model Parameters:")
for param in poly_model.parameters():
    print(param)

Linear Regression Model Parameters:
Parameter containing:
tensor([[46.3305]], requires_grad=True)
Parameter containing:
tensor([-8.0242], requires_grad=True)

Polynomial Regression Model Parameters:
Parameter containing:
tensor([[ 2.9974, -1.8134,  0.0578,  1.1134]], requires_grad=True)
Parameter containing:
tensor([1.1290], requires_grad=True)


In [9]:
# Plot the results using Plotly
X_space = torch.arange(-5, 5, 0.001, dtype=torch.float32).view(-1, 1)

predicted_linear = linear_model(X_space).detach().numpy()
predicted_poly = poly_model(X_space).detach().numpy()

scatter = go.Scatter(x=X, y=y, mode='markers', name='True data')
linear_line = go.Scatter(x=X_space.numpy().flatten(), y=predicted_linear.flatten(), mode='lines', name='Linear model', line=dict(color='green'))
poly_line = go.Scatter(x=X_space.numpy().flatten(), y=predicted_poly.flatten(), mode='lines', name='Poly model', line=dict(color='red'))

layout = go.Layout(
    title="Linear and Poly Regression Fit Compare",
    xaxis=dict(title="X"),
    yaxis=dict(title="y"),
    showlegend=True
)

fig = go.Figure(data=[scatter, linear_line, poly_line], layout=layout)
fig.show()
